# Lending Club Data - Data Prep for H2O.ai Auto ML

**Lending Club** publishes loan data periodically for public use. The anonymized data has information on different loans issued, such as loan_id, loan_amount, what loan grade, credit score etc., and also a **loan_status** variable that indicates whether a loan was _Fully Paid_ ,_Charged Off_ (written off because of default), _Late__by__X__months_ etc., 

This python notebook tutorial shows:
- How to get the data from Lending Club site
- Keep rows for only two classes - to make it a simple binary classification problem
- Drop some Target Leakage Columns
- Create a train and test data set


### Import numpy, pandas, sklearn etc., to do the data munging


In [1]:
import numpy as np
import pandas as pd

import requests
import math
from sklearn import model_selection


### Download 2014 Loan Data from Lending Club Website

The url provided in the wget command is where the 2014 loan data is. You can also get loan data over different periods from https://www.lendingclub.com/info/download-data.action 



In [2]:
!rm LoanStats3c.csv.zip
!wget --no-check-certificate "https://resources.lendingclub.com/LoanStats3c.csv.zip"

rm: cannot remove 'LoanStats3c.csv.zip': No such file or directory
--2021-06-19 15:01:16--  https://resources.lendingclub.com/LoanStats3c.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 35.164.161.63, 44.236.155.144, 44.236.27.127
Connecting to resources.lendingclub.com (resources.lendingclub.com)|35.164.161.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats3c.csv.zip’

LoanStats3c.csv.zip     [ <=>                ]  37.20M  5.72MB/s    in 6.9s    

2021-06-19 15:01:24 (5.36 MB/s) - ‘LoanStats3c.csv.zip’ saved [39009667]



### Data Dictionary - for Lending Club Data
We next display the data dictionary of the Lending Club Data. 

Note: Though there are 153 columns in the data dictionary, the actual data (in the next section) has only 145 columns

In [ ]:
#!pip install xlrd  # Uncomment this, if you get errors parsing XLSX
pd.set_option('expand_frame_repr', True)
pd.set_option('max_colwidth',9999)
pd.set_option('display.max_columns',9999)
pd.set_option('display.max_rows',9999)
data_dictionary = pd.read_excel("https://resources.lendingclub.com/LCDataDictionary.xlsx")

data_dictionary

### Munging the data in Pandas

We first read the data into a Pandas data frame. We skip reading the very first row as it has some non-header information. The target column **loan_status** has really 4 unique values, but we keep only the rows with **Fully Paid** and **Charged Off** status, to make it into a binary classification problem.

In [16]:
# Very first row has non-header data and hence skipping it. Read to a data frame
# Fix the Mon-Year on one column to be readable

def parse_dates(x):
    return datetime.strptime(x, "%b-%d")

lc = pd.read_csv("LoanStats3c.csv.zip", skiprows=1,verbose=False, parse_dates=['issue_d'],low_memory=False) 
lc.shape

(235631, 144)

In [17]:
lc.loan_status.unique()

array(['Fully Paid', 'Charged Off', nan], dtype=object)

In [18]:
# Keep just "Fully Paid" and "Charged Off" to make it a simple 'Yes' or 'No' - binary classification problem

lc = lc[lc.loan_status.isin(['Fully Paid','Charged Off'])]
lc.loan_status.unique()

array(['Fully Paid', 'Charged Off'], dtype=object)

In [19]:
# Drop the columns from the data frame that are Target Leakage ones
# Target Leakage columns are generally created in hindsight by analysts/data engineers/operations after an outcome 
# was detected in historical data. If we don't remove them now, they would climb to the top of the feature list after a model is built and 
# falsely increase the accuracy to 95% :) 
#
# In Production or real life scoring environment, don't expect these columns to be available at scoring time
# , that is,when someone applies for a loan. So we don't train on those columns ...

ignored_cols = [ 
                'out_prncp',                 # Remaining outstanding principal for total amount funded
                'out_prncp_inv',             # Remaining outstanding principal for portion of total amount 
                                             # funded by investors
                'total_pymnt',               # Payments received to date for total amount funded
                'total_pymnt_inv',           # Payments received to date for portion of total amount 
                                             # funded by investors
                'total_rec_prncp',           # Principal received to date 
                'total_rec_int',             # Interest received to date
                'total_rec_late_fee',        # Late fees received to date
                'recoveries',                # post charge off gross recovery
                'collection_recovery_fee',   # post charge off collection fee
                'last_pymnt_d',              # Last month payment was received
                'last_pymnt_amnt',           # Last total payment amount received
                'next_pymnt_d',              # Next scheduled payment date
                'last_credit_pull_d',        # The most recent month LC pulled credit for this loan
                'settlement_term',           # The number of months that the borrower will be on the settlement plan
                'settlement_date',           # The date that the borrower agrees to the settlement plan
                'settlement_amount',         # The loan amount that the borrower has agreed to settle for
                'settlement_percentage',     # The settlement amount as a percentage of the payoff balance amount on the loan
                'settlement_status',         # The status of the borrower’s settlement plan. Possible values are: 
                                             # COMPLETE, ACTIVE, BROKEN, CANCELLED, DENIED, DRAF
                'debt_settlement_flag',      # Flags whether or not the borrower, who has charged-off, is working with 
                                             # a debt-settlement company.
                'debt_settlement_flag_date'  # The most recent date that the Debt_Settlement_Flag has been set
                ]

lc = lc.drop(columns=ignored_cols, axis = 1)

In [20]:
# After dropping Target Leakage columns, we have 223K rows and 125 columns
lc.shape

(235629, 124)

### Split to Train and Test - 80%/20%  and write it to two zipped CSV files

In [21]:
import csv
import os 

train_path = os.getcwd() + "/train_lc.csv.zip"
test_path = os.getcwd() + "/test_lc.csv.zip"
print(lc.shape)
train_lc, test_lc = model_selection.train_test_split(lc, test_size=0.2, random_state=10,stratify=lc['loan_status'])
train_lc.to_csv(train_path, index=False,compression="zip")
test_lc.to_csv(test_path, index=False,compression="zip")


(235629, 124)


In [22]:
# Let's look at the shape of training data
train_lc.shape

(188503, 124)

In [ ]:
# Let's look at the shape of test data

In [23]:
test_lc.shape

(47126, 124)

In [ ]:
# In the next notebook(s) we will use the training data set to use Automatic Machine Learning to build AI/ML models 
# and will try to predict on the test data set and evaluate the results!

# Lending Club Data - Auto ML on H2O-3 Open Source

**Lending Club** publishes loan data periodically for public use. The anonymized data has information on different loans issued, such as loan_id, loan_amount, what loan grade, credit score etc., and also a **loan_status** variable that indicates whether a loan was _Fully Paid_ or _Charged Off_ (written off because of default).

This python notebook tutorial shows:
- Kick off Automatic Machine Learning with H2O3 with the training/test prepared in the previous tutorial
- Show Variable Importance
- Score on test data
- Show model performance on test data

Prerequisites:
- A H2O3 instance available running on-prem or cloud. If not, you can start a local instance
- Get the latest h2o python library by running "!pip install h2o" or "!conda install h2o" etc.,

In [4]:
# Replace below with settings in your environment before you run the notebook

h2o3_ip = "127.0.0.1"
 


## Create or Connect to H2O3 instance 


In [5]:
!pip install h2o 
import h2o
from h2o.automl import H2OAutoML

# If you want to create and run a local instance instead, (where Jupyter kernel is running) just uncomment below:
#h2o.init()
# and comment the code below:

h2o.init(ip=h2o3_ip,strict_version_check=True)

# If things break, uninstall and install the h2o library and you set strick_version_check to True to see 
# what's going on

Checking whether there is an H2O instance running at http://127.0.0.1:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5gha37xi
  JVM stdout: /tmp/tmp5gha37xi/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp5gha37xi/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,30 days
H2O_cluster_name:,H2O_from_python_unknownUser_tz28fp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.174 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


### Upload the Train and Test data to H2O-3 Cluster

In [ ]:
import os

# These two CSV files were created in the previous section

train_path = os.getcwd()+"/train_lc.csv.zip"
test_path = os.getcwd()+ "/test_lc.csv.zip"

train = h2o.load_dataset(train_path)
test = h2o.load_dataset(test_path)



In [ ]:
train.describe()


### Run Auto ML on training data set


In [ ]:
# Identify predictors and response
x = train.columns
y = "loan_status"
x.remove(y)


# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML 
aml = H2OAutoML(project_name='LC', 
                max_models=5,         # 50 base models
                balance_classes=True,  # Doing smart Class imbalance sampling
                max_runtime_secs=7200, # 2 hours
                seed=1234)             # Set a seed for reproducability
aml.train(x=x, y=y, training_frame=train)

### Look at AutoML Leaderboard

In [10]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20210619_153156,0.707886,0.426223,0.913134,0.495639,0.365696,0.133734
XGBoost_3_AutoML_20210619_153156,0.705041,0.427452,0.912232,0.495835,0.366287,0.134166
XGBoost_3_AutoML_20210619_151900,0.704968,0.427484,0.912226,0.495616,0.366303,0.134178
GLM_1_AutoML_20210619_153156,0.704136,0.42802,0.911358,0.493174,0.366385,0.134238
XGBoost_1_AutoML_20210619_153156,0.688296,0.436259,0.906032,0.499124,0.370229,0.13707
XGBoost_1_AutoML_20210619_151900,0.687911,0.435824,0.905412,0.499445,0.370072,0.136954
XGBoost_2_AutoML_20210619_153156,0.670303,0.445361,0.898407,0.499928,0.37457,0.140302
XGBoost_2_AutoML_20210619_151900,0.669521,0.445566,0.897438,0.499905,0.37462,0.14034
DRF_1_AutoML_20210619_153156,0.657993,0.670194,0.888488,0.5,0.383728,0.147247


### What's the composition of the StackedEnsemble Leader model ?

In [ ]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
metalearner = h2o.get_model(aml.leader.metalearner()['name'])
metalearner.std_coef_plot()

>  You can see above H2O ML ran XGBoost, GLM, XRT, GBM, Deep Learning, DRF etc., did hyperparameter search and then

>  combined the top N best algorithms to create a final stack ensemble

### Important Features and Attributes of the top GLM Model on the AutoML leaderboard
#### What's different about a  Loan getting "Fully Paid" or "Charged Off" ?

In [ ]:
# Get the GLM leader from above for looking at feature importance
df = lb.as_data_frame()
for m in df['model_id'].tolist():
    if (m.find("GLM") != -1):
        glm_leader_model = m
        break
        
h2o.get_model(glm_leader_model).std_coef_plot(num_of_features=20)

### Insights ...
The standardized Coef. Magnitudes of the GLM model in the leaderboard gives us a sense of what's different about a Loan Getting Paid in Full vs Loan getting Charged Off/Defaulted. The features/attributes in blue are the positive reasons (Length of the bar is the order of importance) why the Loan is getting Paid in Full vs the one in the Orange which can be attributed to Loan defaulting. In summary:

##### 7 Factors why Loan might getting Fully Paid - in the order of importance (Looking only at the Blue bars):

1. **term.36_months** - If the Loan term is shorter, like 3 years
2. **grade.A** - If the Loan Grade is "A"
3. **total_bc_limit** - If the total bank card credit Limit is high
4. **mo_sub_old_rev_tl_op** - If a lot of months since most recent revolving account opened
5. **home_ownership.MORTGAGE** - Whether a Home Mortgage Account was opened earlier by customer
6. **total_il_high_credit_limit** - Total installment high credit/credit limit (Kind of %payments to total credit limit)
7. **earliest_cr_line** - When the first credit line was opened


##### 7 Factors why Loan might be Charged Off - in the order of importance (Looking only at the Orange bars):
1. **int_rate** - If Interest Rate is high
2. **term.60_months** - If the Loan term is longer, like 5 years
3. **emp_title.<EMP_TITLE>** - 
4. **acc_open_past_24_mnts** - Correlated to the # of accounts opened in past 24 months
5. **dti** - Debt to Income ratio is high
6. **issue_d** - month/year which a loan was issued
7. **home_ownership.RENT** - Whether a customer was renting an apartment.

### Variable Importance Insights from each model in the AutoML Leaderboard

Note that each algorithm in the leaderboard is somewhat showing the same variables as the Top N important ones. That's telling us each algorithm generalizes to the same factors more or less !

In [ ]:
# Let's do variable importance for each one of the models in the LeaderBoard
# We can't do this for a StackedEnsemble, so will skip it

df = lb.as_data_frame()
for m in df['model_id'].tolist():
    if (m.find("StackedEnsemble") == -1):
        h2o.get_model(m).varimp_plot()
 

### Lets predict on the Test Data Set - using the AutoML leader 

In [14]:
test_pc = aml.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [15]:
# These scores were predicted by the stacked ensemble leader ...
test_pc

predict,Charged Off,Fully Paid
Fully Paid,0.106697,0.893303
Fully Paid,0.0880756,0.911924
Fully Paid,0.136152,0.863848
Fully Paid,0.181023,0.818977
Fully Paid,0.170059,0.829941
Fully Paid,0.103988,0.896012
Fully Paid,0.0646339,0.935366
Fully Paid,0.0848739,0.915126
Fully Paid,0.18764,0.81236
Fully Paid,0.110748,0.889252




```
# This is formatted as code
```

### AutoML Leader - Model Performance on the Test Data Set

In [ ]:
aml.leader.model_performance(test_data=test,xval=False)

### Summary of Results
The AUC on the test set was ~ 0.73, which is slightly better than the train AUC. 

The data was a snapshot on time where loans where running (some early stage and some late) and not necessarily "cohorts". In the data preparation phase, we also dropped lot of columns that was giving away the outcome. The models built are still very useful to understand the drivers behind the outcome. How to improve the accuracy ? See Next Steps.

### Next Steps
H2O3 AutoML can help you build models really quickly and understand the variable importances with very little effort. Recall, we didn't do any feature engineering (like one-hot-encoding etc.,) to the input data! In the next blog posts, we will explore how to do the following, in addition to Automatic Machine Learning:
- Automatic Feature Engineering
- Machine Learning Interpretability etc.,

with H2O's commercial product Driverless AI ...
